In [1]:
import os
import gdown
import shutil
import yaml
import copy

In [2]:
# Чтение конфигурационного файла
def read_yaml_file(file_path):
    with open(file_path, 'r') as file:
        try:
            data = yaml.safe_load(file)
            return data
        except yaml.YAMLError as e:
            print("Error reading YAML file:", e)

# experiments = read_yaml_file('../configs/VOC_original_py_partial.yaml')


In [3]:
def merge_dict(dict1, dict2):
    dict1_copy = copy.deepcopy(dict1)
    dict2_copy = copy.deepcopy(dict2)
    def _merge_dict(dict1, dict2):
        for key, val in dict1.items():
            if type(val) == dict:
                if key in dict2 and type(dict2[key] == dict):
                    _merge_dict(dict1[key], dict2[key])
            else:
                if key in dict2:
                    dict1[key] = dict2[key]

        for key, val in dict2.items():
            if not key in dict1:
                dict1[key] = val
        return dict1
    return _merge_dict(dict1_copy, dict2_copy)

In [8]:
# Создание словарика конфигурационного файла


label_config_common = {
    'data_config':{
        'dataset_name': 'Vehicle',
        'train_postfix':None, 
        'val_postfix':None,
        'test_postfix':None
    }
}

label_configs = [
        (
        {
            'data_config':{
                'train_postfix':'GD', 
                }
        },
        'GD_train'
    ),
    (
        {
            'data_config':{}
        },
        'orig'
    ),

    # (
    #     {
    #         'data_config':{
    #             'train_postfix':'GD',
    #             'val_postfix':'GD',
    #             }
    #     },
    #     'GD_train_val'
    # ),
]

train_config_common = {
    'train_config':{
                'data': 'datasets/Vehicle/Vehicle_test.yaml',
                'imgsz': 640,
                'workers': 2,
                'optimizer': 'AdamW',
            }
}

test_config_common = {
    'test_config':{
                'data': 'datasets/Vehicle/Vehicle_test.yaml',
                'imgsz': 640
            }
}



model_configs = [
    (
        {
            'model':'yolov8n.pt',
            'train_config':{
                'epochs': 60,
                'batch': 32,
                'lr0': 0.00005,
                'lrf': 0.01,
                'patience': 100,
            },
            'test_config':{'batch': 32}
        },
        'nano'
    ),
    # (
    #     {
    #         'model':'yolov8s.pt',
    #         'train_config':{
    #             'epochs': 50,
    #             'batch': 24,
    #             'lr0': 0.00001,
    #             'lrf': 0.01,
    #             'patience': 100,
    #         },
    #         'test_config':{'batch': 24}
    #     },
    #     'small'
    # ),
    # (
    #     {
    #         'model':'yolov8m.pt',
    #         'train_config':{
    #             'epochs': 70,
    #             'batch': 16,
    #             'lr0': 0.000005,
    #             'lrf': 0.01,
    #             'patience': 150,
    #         },
    #         'test_config':{'batch': 16}
    #     },
    #     'medium'
    # ),
    # (
    #     {
    #         'model':'yolov8l.pt',
    #         'train_config':{
    #             'epochs': 80,
    #             'batch': 12,
    #             'lr0': 0.000003,
    #             'lrf': 0.01,
    #             'patience': 150,
    #         },
    #         'test_config':{'batch': 12}
    #     },
    #     'large'
    # ),
    # (
    #     {
    #         'model':'yolov8x.pt',
    #         'train_config':{
    #             'epochs': 90,
    #             'batch': 8,
    #             'lr0': 0.000003,
    #             'lrf': 0.01,
    #             'patience': 150,
    #         },
    #         'test_config':{'batch': 8}
    #     },
    #     'extra_large'
    # ),

]




In [21]:
#   saving_directory: runs/detect/GD_freeze_backbone
# label_configs
# label_config_common
# train_config_common
# test_config_common
# model_configs

all_configs = []
keyorder = ['model', 'saving_directory', 'data_config', 'train_config', 'test_config']
experiment_name = 'all_models_Vehicle' 

for l_conf in label_configs:
    for m_conf in model_configs:
        conf = {}
        conf = merge_dict(conf, label_config_common)
        conf = merge_dict(conf, train_config_common)
        conf = merge_dict(conf, test_config_common)
        conf = merge_dict(conf, l_conf[0])
        conf = merge_dict(conf, m_conf[0])

        conf['saving_directory'] = f'runs/{experiment_name}/{l_conf[1]}/{m_conf[1]}'
        # sort dict
        conf_sorted = {k: conf[k] for k in keyorder if k in conf}
        all_configs.append(conf_sorted)



In [14]:
!ls ../configs -1

download_GD_annotaion.yaml
old
VOC_all_models.yaml
VOC_freeze.yaml
VOC_split_data_30.yaml
VOC_split_data.yaml
VOC_warmup.yaml


In [23]:
len(all_configs)

10

In [22]:
# all_configs[0]

# Запись в конфигурационный файл
config_filename = '../configs/Vehicle_all_models.yaml' 

for conf in all_configs:
    with open(config_filename, 'a') as outfile:
        yaml.dump([conf], outfile, default_flow_style=False, sort_keys=False)

    with open(config_filename, 'a') as outfile:
        outfile.write('\n\n')



In [5]:
# additional config partitial

    # seed: 3
    # number_samples: 500

split_config = [

    (
        {   
            'data_config':{
                'seed':42, 
                'number_of_samples':100,
            }
        },
        [1,2,3,4,5]
    ),

    (
        {   
            'data_config':{
                'seed':42, 
                'number_of_samples':200,
            }
        },
        [1,2,3,4,5]
    ),

    (
        {   
            'data_config':{
                'seed':42, 
                'number_of_samples':500,
            }
        },
        [1,2,3,4,5]
    ),

    (
        {   
            'data_config':{
                'seed':42, 
                'number_of_samples':1000,
            }
        },
        [1,2,3]
    ),

    (
        {   
            'data_config':{
                'seed':42, 
                'number_of_samples':2000,
            }
        },
        [1]
    ),
    (
        {   
            'data_config':{
                'seed':42, 
                'number_of_samples':3500,
            }
        },
        [1]
    ),

]

In [12]:
#   saving_directory: runs/detect/GD_freeze_backbone
# label_configs
# label_config_common
# train_config_common
# test_config_common
# model_configs

all_configs = []
keyorder = ['model', 'saving_directory', 'data_config', 'train_config', 'test_config']
experiment_name = 'split_exp_vehicle' 

for l_conf in label_configs:
    for m_conf in model_configs:
        for s_conf in split_config:
            for data_split in s_conf[1]:
            

                conf = {}
                conf = merge_dict(conf, label_config_common)
                conf = merge_dict(conf, train_config_common)
                conf = merge_dict(conf, test_config_common)
                conf = merge_dict(conf, l_conf[0])
                conf = merge_dict(conf, m_conf[0])
                conf = merge_dict(conf, s_conf[0])





                split_conf = {
                    'data_config':{
                        'data_split':data_split, 
                    }
                }
                # print(split_conf)
                conf = merge_dict(conf, split_conf)
                # print(conf)
                # break

                s_path = s_conf[0]['data_config']['number_of_samples']


                conf['saving_directory'] = f'runs/{experiment_name}/{l_conf[1]}/{m_conf[1]}/{s_path}/exp_{data_split}'
                # sort dict
    
                conf_sorted = {k: conf[k] for k in keyorder if k in conf}
                # print(conf_sorted)
                all_configs.append(conf_sorted)


In [13]:
len(all_configs)

40

In [11]:
# Запись в конфигурационный файл
config_filename = '../configs/Vehicle_split_data.yaml' 

for conf in all_configs:
    with open(config_filename, 'a') as outfile:
        yaml.dump([conf], outfile, default_flow_style=False, sort_keys=False)

    with open(config_filename, 'a') as outfile:
        outfile.write('\n\n')

In [11]:
conf_sorted['data_config']

{'dataset_name': 'VOC',
 'train_postfix': 'GD',
 'val_postfix': 'GD',
 'test_postfix': None,
 'seed': 42,
 'number_of_samples': 1000,
 'split': 2}

In [15]:
all_configs[-1]['data_config']

{'dataset_name': 'VOC',
 'train_postfix': 'GD',
 'val_postfix': 'GD',
 'test_postfix': None,
 'seed': 42,
 'number_of_samples': 1000,
 'split': 2}